# Задания 3-4. Проблема собственных значений


In [1]:
import warnings
from typing import Any, Union, List, Tuple

import numpy as np
import pandas as pd
from IPython.display import Latex, Markdown, display
from latexifier import latexify

warnings.filterwarnings('ignore')

In [2]:
class matrix(np.ndarray):
    """extends `np.ndarray`"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()

    def norm(self) -> float:
        """Calculates Matrix norm using `np.linalg.norm` with parameter `'fro'`"""
        return np.linalg.norm(self, 'fro')

    def inv(self) -> Any:
        return matrix(np.linalg.inv(self))


In [3]:
def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg,
                   np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_exact_column(col: matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of `col` elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


## Матрица: Вариант 12


In [4]:
n = 3
A = matrix([
    [-1.71126, -0.07170, 1.23615],
    [-0.07170, 1.34019, 0.02903],
    [1.23615, 0.02903, -1.71295]
])
print_tex('A=', A)


$$ A= \begin{pmatrix} -1.711260 & -0.071700 & 1.236150 \\ -0.071700 & 1.340190 & 0.029030 \\ 1.236150 & 0.029030 & -1.712950 \end{pmatrix} $$

In [5]:
def visualize_results(i: int, val: float, vec: matrix, A: matrix = A, n: int = n) -> None:
    print_tex(f'\lambda_{i} = ', val, f',~~ x_{i} = ', vec)
    print_tex(f'A x_{i}=', get_exact_column(A @ vec, n), r'\stackrel{?}{=}',
          get_exact_column(val * vec, n), f'=\lambda_{i} x_{i}')

intermediate_results = {}

def put_intermediate_result(name: str, val: float, col: matrix) -> None:

    if name not in intermediate_results:
        intermediate_results[name] = {
            'Intermediate Values': [],
            'Intermediate Columns': []
        }
        
    vals = 'Intermediate Values'
    cols = 'Intermediate Columns'
    
    intermediate_results[name][vals].append(val)
    if len(intermediate_results[name][vals]) == 4:
        intermediate_results[name][vals].pop(0)

    intermediate_results[name][cols].append(col / col.norm())
    if len(intermediate_results[name][cols]) == 4:
        intermediate_results[name][cols].pop(0)


## Задание: Вариант 4


Найдем методом Якоби все собственные числа и собственные векторы с точностью $\varepsilon = 0.000001$. Собственные вектора должны иметь единичную длину.


In [24]:
eps = 1.e-6


def jacobi(A: matrix = A, n: int = n, eps: float = eps) -> Tuple[List[float], List[matrix]]:
    # матрица, столбцы которой будут собственными векторами
    X = matrix(np.identity(n))
    # A^(k)
    A_ = A.copy()

    iter = 0

    while True:
        flat_index = np.argmax(([[abs(A_[i][j]) if j > i else -1 for j in range(n)] for i in range(n)]))
        # тут важно проставлять на j > i число меньше 0, потому что иначе при последней итерации оно умрет
        i_, j_ = flat_index // n, flat_index % n

        if abs(A_[i_][j_]) <= eps:
            break

        d = np.sqrt((A_[i_][i_] - A_[j_][j_])**2 + 4 * A_[i_][j_]**2)

        c = np.sqrt(1 / 2 * (1 + abs(A_[i_][i_] - A_[j_][j_]) / d))

        s = np.sign(A_[i_][j_] * (A_[i_][i_] - A_[j_][j_])) * \
            np.sqrt(1 / 2 * (1 - abs(A_[i_][i_] - A_[j_][j_]) / d))

        A_next = A_.copy()

        for i in range(n):
            if i_ != i != j_:
                A_next[i][i_] = c * A_[i][i_] * s * A_[i][j_]
                A_next[i_][i] = A_next[i][i_]
                A_next[i][j_] = -s * A_[i][i_] + c * A_[i][j_]
                A_next[j_][i] = A_next[i][j_]

            # for j in range(n):
            #     if i != i_ and i != j_ and j != i_ and j != j_:
            #         continue
            #     elif i != i_ and i != j_:
            #         A_next[i][i_] = c * A_[i][i_] * s * A_[i][j_]
            #         A_next[i_][i] = A_next[i][i_]
            #         A_next[i][j_] = -s * A_[i][i_] + c * A_[i][j_]
            #         A_next[j_][i] = A_next[i][j_]

        A_next[i_][i_] = c**2 * A_[i_][i_] + 2 * \
            c * s * A_[i_][j_] + s**2 * A_[j_][j_]
        A_next[j_][j_] = s**2 * A_[i_][i_] - 2 * \
            c * s * A_[i_][j_] + c**2 * A_[j_][j_]
        A_next[i_][j_] = 0
        A_next[j_][i_] = 0

        A_ = A_next.copy()

        V = matrix(np.zeros((n, n)))
        for i in range(n):
            if i != i_ and i != j_:
                V[i][i] = 1
        V[i_][i_] = c
        V[j_][j_] = c
        V[i_][j_] = -s
        V[j_][i_] = s

        # print_tex(X, V, X @ V, i_, '~', j_, '~', c, '~', s)

        X = X @ V  # TODO: можно упростить, так как V - ортоганальная, а это что-то значит

        iter += 1

    eigenvals = [A_[i][i] for i in range(n)]
    # уточним занчения собственных чисел
    for i in range(n):
        eigenvals[i] += np.sum(np.sum([A_[i][j] / (A_[i][i] - A_[j][j])
                                       for j in range(n) if j != i]) for i in range(n))

    return eigenvals, [matrix(row).reshape(n, 1) for row in X.transpose()]


jacobi_vals, jacobi_vecs = jacobi(A, n, eps)

for i in range(n):
    jacobi_vecs[i] /= jacobi_vecs[i].norm()

for i in range(n):
    visualize_results(i+1, jacobi_vals[i], jacobi_vecs[i])

$$ \lambda_1 =  -0.47595471106935777 ,~~ x_1 =  \begin{pmatrix} 0.707343 \\ -0.000000 \\ 0.706870 \end{pmatrix} $$

$$ A x_1= \begin{pmatrix}
-0.33665125090951264\\
-0.030196247047396166\\
-0.3364503224038047
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.3366634632623245\\
5.5181796674541806e-08\\
-0.33643810648220324
\end{pmatrix} =\lambda_1 x_1 $$

$$ \lambda_2 =  1.3413723396695043 ,~~ x_2 =  \begin{pmatrix} -0.011734 \\ 0.999862 \\ 0.011742 \end{pmatrix} $$

$$ A x_2= \begin{pmatrix}
-0.03709593674267801\\
1.3411875182711082\\
-0.005591839725547202
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.015739370255827022\\
1.3411875182711082\\
0.01575013295740321
\end{pmatrix} =\lambda_2 x_2 $$

$$ \lambda_3 =  -2.9494376286001462 ,~~ x_3 =  \begin{pmatrix} -0.706773 \\ -0.016600 \\ 0.707246 \end{pmatrix} $$

$$ A x_3= \begin{pmatrix}
2.084924057810798\\
0.048960131719981856\\
-2.085635916880715
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
2.0845817154195596\\
0.048959920817116594\\
-2.085978025506137
\end{pmatrix} =\lambda_3 x_3 $$

In [25]:
vals, vecs = np.linalg.eigh(A)
vecs = [matrix(row).reshape(n, 1) for row in vecs.transpose()]
# for i in range(n):
#     print_tex(f'A x_{i+1}=', get_exact_column(A @ vecs[i], n), r'\stackrel{?}{=}',
#               get_exact_column(vals[i] * vecs[i], n), f'=\lambda_{i+1} x_{i+1}')
data = {
    'Expected': {},
    'Actual': {},
    'Error': {}
}

vals_, vecs_ = [], []
for i in range(n):
    diff = [abs(jacobi_vals[i] - vals[j]) for j in range(n)]
    index = diff.index(min(diff))
    vals_.append(vals[index])
    vecs_.append(vecs[index])
vals, vecs = vals_.copy(), vecs_.copy()

for i in range(n):
    
    val_key = f'Value {i+1}'
    vec_key = f'Vector {i+1}'
    jacobi_vec = jacobi_vecs[i] * (1 if i != 0 else -1) # это я проставляю, чтобы погрешность правильно считалась для вектора

    data['Expected'][val_key] = str(vals[i])
    data['Actual'][val_key] = str(jacobi_vals[i])
    data['Error'][val_key] = str(abs(jacobi_vals[i] - vals[i]))

    data['Expected'][vec_key] = '-'
    data['Actual'][vec_key] = '-'
    data['Error'][vec_key] = str((jacobi_vec - vecs[i]).norm())

df = pd.DataFrame(data)
df

,Expected,Actual,Error
Value 1,-0.47645607756119285,-0.47595471106935777,0.0005013664918350758
Vector 1,-,-,0.016608498661197197
Value 2,1.3418738076297823,1.3413723396695043,0.0005014679602779459
Vector 2,-,-,0.01660734576273464
Value 3,-2.9494377300685892,-2.9494376286001462,1.0146844298120072e-07
Vector 3,-,-,0.00019568999733710312


Найдем степенным методом с точностью $\varepsilon = 0.001$ максимальное по модулю собственное число $\lambda_1$ и соответствующий ему собственный вектор $x_1$


In [26]:
def apostr_est(Y: matrix, val: float, A: matrix = A) -> float:
    return (A @ Y - val * Y).norm() / Y.norm()


eps = 1.e-3


def power_method(A: matrix = A, eps: float = eps, n: int = n) -> Tuple[float, matrix, int]:
    val = 1
    Y = matrix(np.ones((n, 1)))
    put_intermediate_result('Power Method', val, Y)
    iter = 0

    while apostr_est(Y, val, A) >= eps:
        Y, val = A @ Y, (A @ Y)[0][0] / Y[0][0]
        iter += 1
        put_intermediate_result('Power Method', val, Y)

    Y /= Y.norm()

    return val, Y, iter


power_val, power_vec, power_iter = power_method(A, eps)

visualize_results(1, power_val, power_vec)


$$ \lambda_1 =  -2.949374915929692 ,~~ x_1 =  \begin{pmatrix} 0.706906 \\ 0.016802 \\ -0.707108 \end{pmatrix} $$

$$ A x_1= \begin{pmatrix}
-2.0849962235559145\\
-0.04869512888923552\\
2.0855699888413057
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-2.084931617666833\\
-0.04955432638611253\\
2.08552557239293
\end{pmatrix} =\lambda_1 x_1 $$

In [27]:
diff = [abs(power_val - vals[j]) for j in range(n)]
index_of_max = diff.index(min(diff))

def get_data_frame(val: float, vec: matrix, iter: int, i: int) -> pd.DataFrame:
    df = pd.DataFrame({
        'Value': {
            'Actual': str(val),
            'Expected': str(vals[i]),
            'Error': str(abs(val - vals[i])),
            'Iterations': iter
        },
        'Vector': {
            'Actual': '-',
            'Expected': '-',
            'Error': str((vec - vecs[i]).norm()),
            'Iterations': iter
        }
    }).transpose()

    return df

df = get_data_frame(power_val, -power_vec, power_iter, index_of_max)
df


,Actual,Expected,Error,Iterations
Value,-2.949374915929692,-2.9494377300685892,6.281413889741572e-05,16
Vector,-,-,0.000200519131581504,16


Найдем методом скалярных произведений с точностью $\varepsilon^2 = 0.000001$ максимальное по модулю собственное число $\lambda_1$ и соответствующий ему собственный вектор $x_1$


In [52]:
eps = 1.e-6


def scalar(a: matrix, b: matrix) -> float:
    """`a`, `b` - columns"""
    return np.dot(a[:, 0], b[:, 0])


def scalar_method(A: matrix = A, eps: float = eps, n: int = n) -> Union[float, matrix, int]:
    val = 1
    Y = matrix(np.ones((n, 1)))
    iter = 0
    put_intermediate_result('Scalar Method', val, Y)

    while apostr_est(Y, val, A) >= eps:
        Y_ = A @ Y
        # TODO: в методичке написано, что так надо делать, но та кне надо делать.
        # Y_ /= Y_[np.argmax(np.abs(Y_))]
        val = scalar(Y_, Y) / scalar(Y, Y)
        Y = Y_.copy()
        iter += 1
        put_intermediate_result('Scalar Method', val, Y)

    Y /= Y.norm()

    return val, Y, iter

scalar_val, scalar_vec, scalar_iter = scalar_method(A, eps)

visualize_results(1, scalar_val, scalar_vec)


$$ \lambda_1 =  -2.9494377300680084 ,~~ x_1 =  \begin{pmatrix} -0.706911 \\ -0.016601 \\ 0.707108 \end{pmatrix} $$

$$ A x_1= \begin{pmatrix}
2.084989943049324\\
0.048964351723489515\\
-2.0855700307578817
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
2.0849899599203074\\
0.04896363330393178\\
-2.085570030757519
\end{pmatrix} =\lambda_1 x_1 $$

In [30]:
df = get_data_frame(scalar_val, scalar_vec, scalar_iter, index_of_max)
df

,Actual,Expected,Error,Iterations
Value,-2.9494377300680084,-2.9494377300685892,5.808686864838819e-13,25
Vector,-,-,1.6745874040465853e-07,25


Найдем используя метод скалярных произведений противоположную границу спектра с точностью $\varepsilon = 0.001$

In [32]:
eps = 1.e-3


def opposite_boundary_scalar(
    A: matrix = A,
    eps: float = eps,
    n: int = n,
    scalar_val: float = scalar_val
) -> Tuple[float, matrix, int]:
    B = A.copy()
    for i in range(n):
        B[i][i] -= scalar_val

    val, vec, iter = scalar_method(B, eps)
    val += scalar_val

    return val, vec, iter

oppb_val ,oppb_vec, oppb_iter = opposite_boundary_scalar(A, eps)

visualize_results(3, oppb_val, oppb_vec)

$$ \lambda_3 =  1.341873032694492 ,~~ x_3 =  \begin{pmatrix} -0.023212 \\ 0.999731 \\ 0.000266 \end{pmatrix} $$

$$ A x_3= \begin{pmatrix}
-0.03163064194463675\\
1.3415008681824439\\
-0.0001266567539043981
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.03114704678384524\\
1.3415114497696217\\
0.00035705232927683796
\end{pmatrix} =\lambda_3 x_3 $$

In [33]:
diff = [abs(oppb_val - vals[j]) for j in range(n)]
index_of_min = diff.index(min(diff))
df = get_data_frame(oppb_val, oppb_vec, oppb_iter, index_of_min)
df

,Actual,Expected,Error,Iterations
Value,1.341873032694492,1.3418738076297823,7.749352901598172e-07,15
Vector,-,-,0.0003762073710709438,15


Найти собственное число $\lambda_2$ с точностью $\varepsilon = 0.001$ методом Виландта, используя степенной метод

In [70]:
eps = 1.e-3

def append(M: np.ndarray, N: np.ndarray) -> matrix:
    """Appends two numpy arrays with axis=1"""
    return matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def Gauss(A: matrix, B: matrix, n: int) -> matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return matrix(np.array(X))


def vilandt(A: matrix = A, eps: float = eps, n: int = n) -> Tuple[float, matrix, int]:
    val = -0.5
    val_ = 0
    Y = matrix(np.ones((n, 1)))
    W = A.copy()
    iter =  0

    while abs(val_ - val) >= eps:
        for i in range(n):
            W[i][i] = A[i][i] - val
        Y_ = Gauss(W, Y, n)
        mu = scalar(Y_, Y) / scalar(Y, Y)
        val, val_ = val_, 1 / mu + val
        Y_ /= Y_.norm()
        Y = Y_.copy()
        iter += 1

    return val, Y, iter
        
vil_val, vil_vec, vil_iter = vilandt(A, eps)

visualize_results(2, vil_val, vil_vec)

$$ \lambda_2 =  -0.47650520761625187 ,~~ x_2 =  \begin{pmatrix} 0.706912 \\ 0.016620 \\ 0.707106 \end{pmatrix} $$

$$ A x_2= \begin{pmatrix}
-0.33681330238385176\\
-0.00788496257878708\\
-0.33690506241758666
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.33684738065450753\\
-0.007919304658294175\\
-0.3369396676637136
\end{pmatrix} =\lambda_2 x_2 $$

In [71]:
diff = [abs(vil_val - vals[j]) for j in range(n)]
index_of_mid = diff.index(min(diff))
df = get_data_frame(vil_val, -vil_vec, vil_iter, index_of_mid) # тут '-' ставитсся, чтобы погрешность правильно считалась
df

,Actual,Expected,Error,Iterations
Value,-0.47650520761625187,-0.47645607756119285,4.913005505902124e-05,3
Vector,-,-,1.8441058866620703e-05,3


Применим уточнение по Эйткену

In [88]:
# TODO: фиксить надо Эйткена, потому что он не работает нормально
def aitken(
    vals: List[float],
    vecs: List[matrix],
    n: int = n
) -> Tuple[float, matrix]:
    val1, val2, val3 = vals
    val = (val3 * val1 - val2**2) / (val3 - 2 * val2 + val1)

    vec1, vec2, vec3 = vecs
    vec = matrix(np.zeros((n, 1)))
    for i in range(n):
        vec[i][0] = (vec3[i][0] * vec1[i][0] - vec2[i][0]**2) / (vec3[i][0] - 2 * vec2[i][0] + vec1[i][0])
    vec /= vec.norm()
    return val, vec

aitken_val, aitken_vec = aitken(
    intermediate_results['Power Method']['Intermediate Values'],
    intermediate_results['Power Method']['Intermediate Columns']
)

visualize_results(1, aitken_val, aitken_vec)


$$ \lambda_1 =  -2.949437732534622 ,~~ x_1 =  \begin{pmatrix} -0.023681 \\ 0.999720 \\ 0.000102 \end{pmatrix} $$

$$ A x_1= \begin{pmatrix}
-0.031028461555907903\\
1.3415150699202358\\
-0.00042720185841563273
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.06984643385247842\\
-2.948610574915921\\
-0.00030211101531741745
\end{pmatrix} =\lambda_1 x_1 $$

In [83]:
df = get_data_frame(aitken_val, aitken_vec, '-', index_of_max)
df

,Actual,Expected,Error,Iterations
Value,-2.949437732534622,-2.9494377300685892,2.466032711367916e-09,-
Vector,-,-,1.41406049227916,-
